In [1]:
from gensim.test.utils import datapath
from gensim.models import FastText
import pandas as pd
import nltk
from gensim.parsing.preprocessing import STOPWORDS
import re
import cupy as cp
import spacy
import cudf
import numpy as np
nltk.download('stopwords') 
nltk.download('wordnet') 

/opt/conda/lib/python3.10/site-packages/cudf/utils/_numba.py:110: UserWarning: Using CUDA toolkit version (12, 4) with CUDA driver version (12, 2) requires minor version compatibility, which is not yet supported for CUDA driver versions 12.0 and above. It is likely that many cuDF operations will not work in this state. Please install CUDA toolkit version (12, 2) to continue using cuDF.
  warnings.warn(
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.NativeFile size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.B

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
STOPWORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [3]:
data_tip=pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json', lines=True,)[:50]


In [4]:
tip_text = data_tip['text']

In [5]:
tip_text[:3]

0                       Avengers time with the ladies.
1    They have lots of good deserts and tasty cuban...
2               It's open even when you think it isn't
Name: text, dtype: object

In [6]:
import spacy

def process_text(document,stopwords=STOPWORDS):
    nlp = spacy.load("en_core_web_sm")
    document = re.sub(r'\s+', ' ', document, flags=re.I) 
    document = re.sub(r'\W', ' ', str(document)) 
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) 
    document = document.lower() 
    doc_spacy=nlp(document)
    lemma_txt = ["".join(token.lemma_) for token in doc_spacy]
    lemma_no_stop_txt = [word for word in lemma_txt if word not in STOPWORDS]
    lemma_no_stop_txt = [word for word in lemma_no_stop_txt if len(word) > 3]
    clean_txt = ' '.join(lemma_no_stop_txt)
    return lemma_no_stop_txt

tip_text_preprocessed = tip_text.apply(process_text)



In [7]:
tip_text_preprocessed

0                                 [avenger, time, lady]
1                [good, desert, tasty, cuban, sandwich]
2                                         [open, think]
3                              [decent, fried, chicken]
4                  [appetizer, platter, special, lunch]
5     [chili, single, cheeseburger, onion, pickle, r...
6     [saturday, 2013, ride, patco, silver, sleigh, ...
7     [probably, good, place, cool, spring, area, wa...
8                                               [tacos]
9       [starbuck, substitute, boring, downtown, tampa]
10                              [order, tortilla, soup]
11                             [good, definitely, come]
12                                     [hotlight, stop]
13                                             [yankee]
14                             [basically, food, rally]
15    [dinner, close, yvonne, negative, star, restau...
16                                [right, right, tight]
17    [grill, chicken, entree, delicious, small,

In [8]:
model3= FastText(vector_size=50,workers=20,window=3,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=1000)

In [9]:
model3.wv.vectors_vocab 

array([[ 0.15335172,  0.58821344,  0.30430937, ...,  0.04975872,
        -0.5192971 ,  0.0720299 ],
       [-0.43196577,  0.32119054,  0.6318146 , ..., -0.12445339,
        -0.06785066, -0.16140959],
       [-0.15478744,  0.4927729 ,  0.6094719 , ...,  0.2843802 ,
         0.01366221,  0.23074089],
       ...,
       [ 0.37163493,  0.03581389,  0.3593771 , ..., -0.0525254 ,
         0.28422394,  0.29095733],
       [ 0.4566189 ,  0.0715327 ,  0.4561432 , ..., -0.00798853,
         0.2976988 ,  0.4228544 ],
       [ 0.24755485,  0.6948096 ,  0.80767107, ..., -0.05976299,
        -0.05170526, -0.30345526]], dtype=float32)

In [10]:
model3.ns_exponent #PROBABILITY OF NEGATIVE WORD SAMPLING

0.75

In [11]:
similar_words = model3.wv.most_similar(positive=['cat'], topn=10)
print(similar_words)


[('castle', 0.8580734133720398), ('carl', 0.8387580513954163), ('shuttle', 0.8183960914611816), ('card', 0.8177192807197571), ('applicable', 0.8158126473426819), ('long', 0.8083181977272034), ('menu', 0.8061986565589905), ('coast', 0.7972036004066467), ('turkey', 0.7936736941337585), ('mini', 0.7890226244926453)]


In [12]:
len(model3.wv.key_to_index)

173

In [13]:
sims = model3.wv.most_similar('lots',topn=10) 

In [14]:
sims

[('look', 0.9465135931968689),
 ('mini', 0.9236357808113098),
 ('similar', 0.9112963080406189),
 ('castle', 0.9079959988594055),
 ('burger', 0.892065167427063),
 ('white', 0.8915032744407654),
 ('menu', 0.867596447467804),
 ('long', 0.8670199513435364),
 ('like', 0.8296854496002197),
 ('love', 0.8296142220497131)]